In [1]:
!sudo apt update -y
!wget https://github.com/wkhtmltopdf/packaging/releases/download/0.12.6-1/wkhtmltox_0.12.6-1.bionic_amd64.deb
!sudo apt install ./wkhtmltox_0.12.6-1.bionic_amd64.deb

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,377 kB]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:13 http://ppa.launchpad.

In [2]:
!which wkhtmltopdf

/usr/local/bin/wkhtmltopdf


In [3]:
# !pip install fpdf
# !pip install pypandoc
# !pip install htmldocx
# import pypandoc
# from htmldocx import HtmlToDocx
# from fpdf import FPDF
# from weasyprint import HTML, CSS
# import pdfkit as pdf
# !pip install wkhtmltopdf
# !pip install weasyprint

!pip install jinja2
!pip install pdfkit

import shutil
import pdfkit
config = pdfkit.configuration(wkhtmltopdf='/usr/local/bin/wkhtmltopdf')

from jinja2 import Environment, FileSystemLoader

import smtplib

import os

from datetime import date, timedelta, datetime

from functools import reduce

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# grab work sheet using work sheet ID Key
worksheet = gc.open_by_key('1PMgartubKK_QCB-Rgvh-zMkO-vg2VDclBh1ZfcxMTGE')
worksheet1 = worksheet.get_worksheet(2) #this is the list of student reports
worksheet2 = worksheet.get_worksheet(4) # this is the list of staff names and emails
worksheet3 = worksheet.get_worksheet(3) #this is the admin claim from
worksheet4 = worksheet.get_worksheet(5) #this is the staff contingecy form
worksheet5 = worksheet.get_worksheet(6) #this is the finance form

# get_all_values gives a list of rows.
rows_sw1 = worksheet1.get_all_values() #this is the list of student reports
rows_sw2 = worksheet2.get_all_values() # this is the list of staff names and emails
rows_sw3 = worksheet3.get_all_values() #this is the admin claim from
rows_sw4 = worksheet4.get_all_values() #this is the staff contingecy form
rows_sw5 = worksheet5.get_all_values() #this is the staff contingecy form
# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows_sw1[1:],columns=rows_sw1[0]) #this is the list of student reports
df2 = pd.DataFrame.from_records(rows_sw2[1:],columns=rows_sw2[0]) # this is the list of staff names and emails
df3 = pd.DataFrame.from_records(rows_sw3[1:],columns=rows_sw3[0]) #this is the admin claim from
#df4 = pd.DataFrame.from_records(rows_sw4[1:],columns=rows_sw4[0]) #this is the staff contingecy form
df5 = pd.DataFrame.from_records(rows_sw5[1:],columns=rows_sw5[0]) #this is the finance form




In [5]:
df5.head()

,Student Name,Actual Hrs/wk,Estemated Allowed Hrs/mth,Rate Commissioned,Staff Rate
0,Adham Salem,15,60,35,16.5
1,Alfie Sarjant,0,0,35,16.5
2,Amy Disley,18,72,35,16.5
3,Archie Miles,4,16,35,16.5
4,Archie Morgan,0,0,35,16.5


In [6]:
# create dictionary map object for renaming google form column names
columns = {
    	'Staff Name':'staff_name',
      'Student Name':'student_name', 	
      'Agreed Checkin Time':'agreed_in', 	
      'Actual Checkin Time':'actual_in', 	
      'Agreed Checkout Time':'agreed_out',	
      'Actual Checkout Time':'actual_out',
      'Session Status ':'session_status', 	
      'Session Report':'session_report',	
      'Evidence of work':'Evidence', 	
      'Please provide link to body of work ':'body_of_work',	
      'Reason for session disruption   (If known)':'known_disruption', 	
      'Staff Email': 'staff_email',	
      'EHCP OutCome (Areas Covered)':'EHCP',	
      '': 'confirm',
      'How did you infom your line manager?': 'manager_informed',
}

#renaming of columns of student reports
df.rename(columns=columns,inplace=True)

In [7]:
# create dictionary map object for renaming google form column names
#renaming of columns for the admin claim from
columns = {
    	'Staff Name':'staff_name',		
      'Start Time':'actual_in', 		
      'End Time':'actual_out', 	
      'Nature of Claim ':'session_status',
      'Date of Claim': 'Date'

}

# making agreed and actual values identical for claims form 
df3.rename(columns=columns,inplace=True)
df3['agreed_in'] = df3['actual_in']
df3['agreed_out'] = df3['actual_out']
# df3


In [8]:
# drop columns not needed for staff reports and claim forms
df = df.drop(['session_report','Evidence','body_of_work','EHCP','confirm','Timestamp','known_disruption'],axis=1)
df3 = df3.drop(['Timestamp'],axis=1)


In [9]:
#new dataFrame creared contianing both student repots and claim detials
df = pd.concat([df,df3],ignore_index=True) 

In [10]:
df.head()

,Day,Date,staff_name,student_name,agreed_in,actual_in,agreed_out,actual_out,session_status,staff_email,Nature of Session Disruption,How did you inform your line manager?
0,Wednesday,16/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,chuk.education@bakedbeancharity.com,,
1,Thursday,17/03/2022,CHUKS OREFO,Jay Casey,10:00:00 AM,10:00:00 AM,2:30:00 PM,2:00:00 PM,Session ran as normal,"chris.education@bakedbeancharity.com, chuk.edu...",,
2,Tuesday,15/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,"chuk.education@bakedbeancharity.com, rachel.ed...",,
3,Monday,14/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,"chuk.education@bakedbeancharity.com, rachel.ed...",,
4,Monday,14/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,"chuk.education@bakedbeancharity.com, rachel.ed...",,


In [11]:
# make new date column
# df['new_date'] = df['Date'].astype(str) + '/' + df['Month'].astype(str) + '/' + df['Year'].astype(str)

# asigin newDate as Date column
print(df['Date'])
print(df['staff_name'])
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%d/%m/%Y')

# make new dateTime string columns
#df['agreed_in_date'] = df['Date'].astype(str) + ' ' + df.agreed_in checked 18/08/22
#df['agreed_out_date'] = df['Date'].astype(str) + ' ' + df.agreed_out checked 18/08/22

df['actual_in_date'] = df['Date'].astype(str) + ' ' + df.actual_in 
df['actual_out_date'] = df['Date'].astype(str) + ' ' + df.actual_out

# convert dateTime strings to dateTime columns
#df['agreed_in_date'] = pd.to_datetime(df['agreed_in_date']) checked 18/08/22
#df['agreed_out_date'] = pd.to_datetime(df['agreed_out_date']) checked 18/08/22

df['actual_in_date'] = pd.to_datetime(df['actual_in_date']) 
df['actual_out_date'] = pd.to_datetime(df['actual_out_date'])

0       16/03/2022
1       17/03/2022
2       15/03/2022
3       14/03/2022
4       14/03/2022
           ...    
2980     1/15/2023
2981     1/23/2023
2982     1/24/2023
2983     1/25/2023
2984      1/2/2023
Name: Date, Length: 2985, dtype: object
0              CHUKS OREFO
1              CHUKS OREFO
2              CHUKS OREFO
3              CHUKS OREFO
4              CHUKS OREFO
               ...        
2980           CHUKS OREFO
2981    Alejandra Carrillo
2982    Alejandra Carrillo
2983    Alejandra Carrillo
2984       NATHANIEL PAMAH
Name: staff_name, Length: 2985, dtype: object


In [12]:
#calculate actual time per session
df['actual_hours'] = df.actual_out_date.dt.hour - df.actual_in_date.dt.hour
df['actual_minutes'] = (df.actual_out_date.dt.minute - df.actual_in_date.dt.minute)/60
df['actual_minutes'] = df['actual_minutes'].map(lambda x: x if x >=0.01 else 0)
df['actual'] = round(df['actual_hours'] + df['actual_minutes'],3)
# df['actual'] = df['actual_hours'].astype(str) +'.'+ df['actual_minutes'].astype(str)


#calculate agreed time per session
#df['agreed_hours'] = df.actual_out_date.dt.hour - df.actual_in_date.dt.hour
#df['agreed_minutes'] = (df.agreed_out_date.dt.minute - df.agreed_in_date.dt.minute)/60
#df['agreed_minutes'] = df['agreed_minutes'].map(lambda x: x if x >=0.01 else 0)
#df['agreed'] = round(df['agreed_hours'] + df['agreed_minutes'],2)
# df['agreed'] = df['agreed_hours'].astype(str) +'.'+ df['agreed_minutes'].astype(str)



In [13]:
df.drop(
    [#'agreed_hours',
    #  'agreed_minutes',
     'actual_hours',
    #  'actual_minutes',
     'actual_out_date',
     'actual_in_date',
     #'agreed_out_date',
     #'agreed_in_date',
    #  'actual_in',
    #  'actual_out',
    #  'agreed_out',
    #  'agreed_in',
     'staff_email',
     ],axis=1, inplace=True)

In [14]:
df.head()

,Day,Date,staff_name,student_name,agreed_in,actual_in,agreed_out,actual_out,session_status,Nature of Session Disruption,How did you inform your line manager?,actual_minutes,actual
0,Wednesday,16/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,,,0.0,4.0
1,Thursday,17/03/2022,CHUKS OREFO,Jay Casey,10:00:00 AM,10:00:00 AM,2:30:00 PM,2:00:00 PM,Session ran as normal,,,0.0,4.0
2,Tuesday,15/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,,,0.0,4.0
3,Monday,14/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,,,0.0,4.0
4,Monday,14/03/2022,CHUKS OREFO,Isaac Dawson,12:00:00 PM,12:00:00 PM,4:00:00 PM,4:00:00 PM,Session ran as normal,,,0.0,4.0


In [15]:
df5.head()

,Student Name,Actual Hrs/wk,Estemated Allowed Hrs/mth,Rate Commissioned,Staff Rate
0,Adham Salem,15,60,35,16.5
1,Alfie Sarjant,0,0,35,16.5
2,Amy Disley,18,72,35,16.5
3,Archie Miles,4,16,35,16.5
4,Archie Morgan,0,0,35,16.5


In [16]:
#creating dict to store individual rates
rates = dict()

# iterate though student rate data frame and storing  student name and thier rate in dictionary
for idx,row in df5.iterrows():
  name = row['Student Name']
  rate = row['Staff Rate']
  rates[name] = f'{name} @ £{rate}'

#creating function to convert standard student name to name with rate
def get_student_rate(student_name):
  return rates.get(student_name,student_name)

#applying map method with callable to transform  data method
# df['student_name'] = df['student_name'].map(get_student_rate)
# df.head()

In [17]:
#convert the df[actual] and df[agreed] columns from ints to floats
df.actual = df.actual.apply(lambda x: float(x))
#df.agreed = df.agreed.apply(lambda x: float(x)) #checked 18/08/22

In [18]:
new_df = df.groupby(['staff_name','student_name','Date'],as_index=False).head()

In [19]:
df2.columns

Index(['Check in Type', ' Attendance Code', 'Staff Name', 'Staff Email',
       'Student Name', 'Date', 'Month', 'Year', 'Nature of Claim ',
       'Time Claimed', 'Tia'],
      dtype='object')

In [20]:
# creates dataFrame with staff name and staff email only by dropping unneeded columns
email_df = df2.drop([' Attendance Code','Date', 'Month', 'Year','Student Name'],axis=1)
personnel = {}

for index, row in email_df.iterrows():
  personnel[row['Staff Name']] = row['Staff Email']

In [21]:
# email_df.head(3)

In [22]:
# df.head(3)

In [23]:
# list and odms created to store and call timesheet data
List = []
future_Date = {
    'year':date.today().year,
    'month':date.today().month,
    'day': 24,
}
last_Invoice_Date = {
    'year':date.today().year -1, # will need to eventully add conditional for december into january  [list comprehnasion]
    'month':date.today().month + 11,
    'day': 16,
}
print(last_Invoice_Date.get('year'))

#create month objects using the date from datetime module
thisMonthAsNumber = date.today().month
lastInvoiceDate = datetime(year=last_Invoice_Date.get('year'),month=last_Invoice_Date.get('month'),day=last_Invoice_Date.get('day')).date()
futureDate = datetime(year=future_Date.get('year'),month=future_Date.get('month'),day=future_Date.get('day')).date()

# dicMap created to store days of month
get_month = {'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'Augst','9':'September','10':'October','11':'November','12':'December',}

2022


In [24]:
# first forloop iterates though the personnel dicMap 
for key, value in personnel.items():
  # data object dom generated per personnel and appended to List
  data = {
        'staff_Name':key,
        'staff_email':value,
        'description':[],
        'day':[],
        'date':[],
        'today':date.today().strftime('%d/%m/%Y'),
        'month':get_month.get(str(date.today().month)),
        'start_actual':[],
        'end_actual':[],
        #'start_agreed':[], #checked 18/08/22
        #'end_agreed':[], #checked 18/08/22
        'actual':[],
        #'agreed':[], #checked 18/08/22
        'outputs':dict(),
        'total_Actual':None, 
        #'total_Agreed':None, #unchecked 18/08/22
        }
  List.append(data)
# populate dom object data from data frame
  for idx, row in df.iterrows():
    #Date = pd.to_datetime(row.Date).strftime('%d/%m/%Y')
    # print(row.Date,Date, row.staff_name,row.student_name)
    thisDate = datetime(year=int(row.Date[6:]),month=int(row.Date[3:5]),day=int(row.Date[:2])).date()
    if (row.staff_name == key) and ((thisDate <= futureDate) and (thisDate > lastInvoiceDate)): #(Date.month == thisMonthAsNumber): legacy code
      
      if isinstance(row.student_name,float) == True:
        List[-1]['description'].append(f'{row.session_status}')
      else:
        List[-1]['description'].append(f'{row.session_status} with {row.student_name}')
        

      List[-1]['day'].append(f'{row.Day}') 
      List[-1]['date'].append(f'{row.Date}') 
      List[-1]['start_actual'].append(f'{row.actual_in}')
      List[-1]['end_actual'].append(f'{row.actual_out}')
      #List[-1]['start_agreed'].append(f'{row.agreed_in}') #checked 18/08/22
      #List[-1]['end_agreed'].append(f'{row.agreed_out}') #checked 18/08/22
      List[-1]['actual'].append(f'{row.actual}')
      #List[-1]['agreed'].append(f'{row.agreed}')

      if isinstance(row.student_name,float):
        row.student_name = f'{row.session_status}' #was previously given the value'Admin'
      
      row.student_name = get_student_rate(row.student_name)
      if row.student_name in List[-1]['outputs'].keys():
        #accessing last dict object at end of list then accessing that last objects key and appending data to it
        #if the key already exist in the object
        #name = rates.get(row.student_name,row.student_name)
        List[-1]['outputs'][row.student_name].append(row.actual)
        #print(row.actual)
       # print(List[-1]['outputs'][name])
      else:
        #otherwise we create the key and its value is a list/array
          #name = rates.get(row.student_name,row.student_name)
          List[-1]['outputs'][row.student_name] = []
          List[-1]['outputs'][row.student_name].append(row.actual)
          #print(List[-1]['outputs'][name])
#name = rates.get(row.student_name,row.student_name)
#

In [25]:
# iterate though each list to calculate total actural and agreed rate
for idx in range(len(List)):
  if (List[idx]['actual'] != []): #and (List[idx]['agreed'] != []): #if not an empty list condition
    List[idx]['total_Actual'] = reduce(lambda x, y: round(float(x)+float(y),3), List[idx]['actual'])
    #List[idx]['total_Agreed'] = reduce(lambda x, y: round(float(x)+float(y),2), List[idx]['agreed'])
    print(List[idx])

{'staff_Name': 'ANNIE SCOTT', 'staff_email': 'annie.scott@bakedbeancharity.uk', 'description': ['Session ran on time and as normal with Fry Family', 'Session ran on time and as normal with Fry Family', 'Session ran on time and as normal with Fry Family', 'Session ran on time and as normal with Billy Fry', 'Session ran on time and as normal with Sadie Hillman', 'Session ran on time and as normal with Arthur Mathias', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Billy Fry', 'Session was Curtailed or Cancelled with Arthur Mathias', 'Session ran on time and as normal with Arthur Mathias', 'Session ran on time and as normal with Amy Disley', 'Session ran on time and as normal with Am

In [26]:
for idx in range(len(List)):
  for items in List[idx]['outputs'].items():
    y = 0 #initializer
    x = 0 #initializer
    List[idx]['outputs'][items[0]] = reduce(lambda x, y: round(float(x)+float(y),3), items[1]) #updated with round function

    

In [27]:
#Gdrive is mounted to google Colab
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [28]:
# chage diretory to save new timeSheets in the timesheet section gdrive
%cd MyDrive/Alpha_project/Staff/TimeSheets/templates/temp

/gdrive/MyDrive/Alpha_project/Staff/TimeSheets/templates/temp


In [29]:
# timesheet copied to colab cloud folder location from gdrive
%cp /gdrive/MyDrive/Alpha_project/Staff/TimeSheets/templates/timeSheetTemplate.html /timeSheetTemplate.html

In [30]:

timesheetPath = r'/gdrive/MyDrive/Alpha_project/Staff/TimeSheets/StaffTimeSheets/'

In [31]:

def getYearMonth():
  return f'{List[1].get("month")}_{date.today().year}'


pathName = getYearMonth()
source = f'/gdrive/MyDrive/Alpha_project/Staff/TimeSheets/templates/temp/{pathName}/'
destination = f'/gdrive/MyDrive/Alpha_project/Staff/TimeSheets/StaffTimeSheets/{pathName}/'
os.makedirs(source, exist_ok=True)
os.makedirs(destination, exist_ok=True)
os.chdir(pathName)

env = Environment(loader=FileSystemLoader('/'))
template = env.get_template('timeSheetTemplate.html') 

for data in List:
  if (data['actual'] == []):
    continue
  person = {}
  person['name'] = data['staff_Name'],
  person['email'] = data['staff_email'],
  person['today'] = data['today'],
  person['month'] = data['month'],
  person['day'] = data['day'],
  person['date'] = data['date'],
  person['description'] = data['description'],
  person['start'] = data['start_actual'],
  person['end'] = data['end_actual'],
  #person['start_agreed'] = data['start_agreed'],
  #person['end_agreed'] = data['end_agreed'],
  person['actual'] = data['actual'],
  #person['agreed'] = data['agreed'],
  person['total_Actual'] = data['total_Actual'],
  #person['total_Agreed'] = data['total_Agreed'],
  person['outputs'] = data['outputs'],
  person['zip'] = zip(
      person.get('day')[0],
      person.get('date')[0],
      person.get('description')[0],
      person.get('start')[0],
      person.get('end')[0],
      person.get('actual')[0],
      #person.get('agreed')[0],
      )

  html = template.render(data=person)
  htmlFile_name = f'{data.get("staff_Name")}_TimeSheet.html'
  pdfFile_name = f'{data.get("staff_Name")}_TimeSheet.pdf'
  path_name = f'{data.get("month")}_{date.today().year}'
  partialPathHtml = f'{path_name}/{htmlFile_name}'
  partialPathPdf = f'{path_name}/{pdfFile_name}'
  pdfkit.from_string(html, pdfFile_name, configuration=config)


  
allfiles = os.listdir(source)
  
for file_ in allfiles:
  if (file_ == '_TimeSheet.pdf'):
    os.remove(file_)
    continue
  shutil.move(source + file_, destination + file_)
os.rmdir(source)

In [32]:
env_source = '/gdrive/MyDrive/Alpha_project/Staff/TimeSheets/templates/'
envSource = os.listdir(env_source)
env = {'email':None,'pass':None,'accounting':None,}
for file_ in envSource:
  if (file_ == 'env.txt'):
    os.chdir(env_source)
    with open(file_,'r',encoding='utf-8') as f:
      env['email'] = f.readline()
      env['pass'] = f.readline()
      env['accounting'] = f.readline()

In [33]:
folder = os.listdir(destination)
collection = {
    'name':[],
    'email':[],
    'pdf':[]
}

for key, value in personnel.items():
  for pdf in folder:
    if key != '':
      if (key in pdf):
        if(key not in collection['name']):
          collection['name'].append(key)
          collection['email'].append(value)
          collection['pdf'].append(pdf)

print(f'list of staff who have completed reports for {getYearMonth().replace("_", " ")}\n\n')
for i in range(len(collection['name'])):
  print(collection['name'][i],collection['email'][i],collection['pdf'][i])


list of staff who have completed reports for January 2023


ANNIE SCOTT annie.scott@bakedbeancharity.uk ANNIE SCOTT_TimeSheet.pdf
CHRIS CROOKS chris.crooks@bakedbeancharity.uk CHRIS CROOKS_TimeSheet.pdf
CHUKS OREFO chuks.orefo@bakedbeancharity.uk CHUKS OREFO_TimeSheet.pdf
FRANCK ZAH franck.zah@bakedbeancharity.uk FRANCK ZAH_TimeSheet.pdf
GLORIA SUNGU KPALABOKO gloria.sungu@bakedbeancharity.uk GLORIA SUNGU KPALABOKO_TimeSheet.pdf
JIM HAWKINS jim.hawkins@bakedbeancharity.uk JIM HAWKINS_TimeSheet.pdf
KITTI WELLS kitti.wells@bakedbeancharity.uk KITTI WELLS_TimeSheet.pdf
KWABENA GYAMFI (Cobbs) cobbs.gyamfi@bakedbeancharity.uk KWABENA GYAMFI (Cobbs)_TimeSheet.pdf
LYN EDWARDS lyn.edwards@bakedbeancharity.uk LYN EDWARDS_TimeSheet.pdf
NAOMI SIMPER naomi.simper@bakedbeancharity.uk NAOMI SIMPER_TimeSheet.pdf
NATHANIEL PAMAH nathaniel.pamah@bakedbeancharity.uk NATHANIEL PAMAH_TimeSheet.pdf
PANAGIOTIS KALOGIROS panagiotis.kalogiros@bakedbeancharity.uk PANAGIOTIS KALOGIROS_TimeSheet.pdf
REGINALD HAY

In [34]:
exclude = []
os.chdir(destination)
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


send = input('enter  "y" for yes when you are ready to send emails to staff:' )
while send != 'y':
  send = input('enter yes if you are read to send email to staff:' )
  print('hello')
for i in range(len(collection['name'])):
  name = collection['name'][i]
  email = collection['email'][i]
  pdf = collection['pdf'][i]

  if(name not in exclude):
    msg = MIMEMultipart()
    msg['Subject'] = f"Your TimeSheet for {getYearMonth().replace('_', ' ')}"
    msg['From'] = env.get("email")
    
    print(name)
    print(email)
    print(pdf)

    msg['To'] = email
    msg.attach(
          MIMEText(f'''Dear {name[0]+name.lower()[1:]},\n\
          \nPlease find attached to this email your auto generated time sheet for {getYearMonth().replace("_", " ")} .\n\
          \nIf you have any missing hours on your timesheet; please check the following before contacting management:\n\
          \n1) Have you completed all daily reports up to and including the 24th of this month? If not this would explain why you have missing hours.\n\
          \n->If this is the case, please complete all daily reports up to and including the 24th by tomorrow end of day and then let Chuks Orefo know (chuks.orefo@bakedbeancharity.uk).\n\
          \n2) If another EA completed a report a result of a 2:1 you must complete a reference report for the day which simply referes to the origianl report.\n
          \n3) If you noticed any missing hours, negative or zero hours on your timesheet for a given day on your timesheet; it means you completed the hours for one of your daily reports incorrectly. Please let management know of any errors\n
          \n4) If you believe that have completed a daily report but it has not shown up on the time sheet sent to you, it's very likely that you entered the wrong date in your daily report. Please review the report for that day to confirm.
          \nFor all other issues relating to your timesheet please let Chuks know by end of working day for the {getYearMonth().replace("_", " ")}.\n\
          \nRegards,\n\
          \nBaked Bean Charity Robot''','plain'))
    
    with open(pdf,'rb') as f:
      attachment = f.read()
      payload = MIMEBase('application', 'octate-stream', Name=pdf)
      payload.set_payload(attachment)
      encoders.encode_base64(payload)
      payload.add_header('Content-Decomposition', 'attachment', filename=pdf)
      msg.attach(payload)
    
    with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
      smtp.login(env.get("email"),env.get('pass'))
      smtp.send_message(msg)
      




KeyboardInterrupt: ignored

In [ ]:




# # attachment = []
# # for file_ in allfiles:
# #   attachment.append(file_)